In [ ]:
  !nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima_model import ARIMA

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 导入数据

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


这次我们只使用1个监测站的数据，即Bloomsbury，进行单边量自回归。

In [3]:
Bloomsbury=pd.read_csv('/content/drive/My Drive/air_inference/data/Bloomsbury_clean.csv')

In [4]:
Bloomsbury=Bloomsbury[['nox','no2','no','o3','pm2.5','ws','wd','air_temp']]

划分训练集,测试集

In [5]:
def splitData(var,per_test):
    num_test=int(len(var)*per_test)
    train_size=int(len(var)-num_test)
    train_data=var[0:train_size]
    test_data=var[train_size:train_size+num_test]
    return train_data,test_data

下面的字典用于存放不同污染物的index

In [6]:
index_dic={
    'nox':0,
    'no2':1,
    'no':2,
    'o3':3,
    'pm2.5':4    
}

# 对单站点单变量进行预测

In [7]:
def predict_arima_sin(attr):
  #数据集
  dataset=Bloomsbury[attr]

  #划分数据集
  train_data,test_data=splitData(dataset,0.1)

  model=ARIMA(train_data,order=(5,1,1)).fit()        
  y_pred=model.forecast(len(test_data))  


  print('mse:', mean_squared_error(test_data, y_pred[0]))
  print('mae:', mean_absolute_error(test_data, y_pred[0]))

  #预测未来
  fut_pred=96
  y_pred=model.forecast(fut_pred+1)
  y_pred=pd.DataFrame(y_pred[0])
  y_pred.to_csv("/content/drive/My Drive/air_inference/result24/arima_"+attr+".csv",header = None, index = None)

In [8]:
%%time
predict_arima_sin('nox')

mse: 1103.7526806258559
mae: 25.32323559695387
CPU times: user 15.2 s, sys: 9.73 s, total: 24.9 s
Wall time: 13.3 s


In [9]:
%%time
predict_arima_sin('no2')

mse: 124.5136144287543
mae: 9.009209045612717
CPU times: user 14.1 s, sys: 8.64 s, total: 22.7 s
Wall time: 11.8 s


In [10]:
%%time
predict_arima_sin('no')

mse: 224.10201116919112
mae: 11.227594901804112
CPU times: user 14.5 s, sys: 9.09 s, total: 23.5 s
Wall time: 12.2 s


In [11]:
%%time
predict_arima_sin('o3')

mse: 153.10804438906428
mae: 9.97472590039405
CPU times: user 14.5 s, sys: 8.9 s, total: 23.4 s
Wall time: 12.1 s


In [12]:
%%time
predict_arima_sin('pm2.5')

mse: 33.25185023551172
mae: 4.264388160147856
CPU times: user 16.1 s, sys: 10.2 s, total: 26.3 s
Wall time: 13.6 s
